In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import math

In [2]:
data = pd.read_csv('sales_history.csv', sep=';')
sample_submission = pd.read_csv('sales_prediciton.csv', sep=';')

In [3]:
data['date'] = data['date'].apply( lambda x: dt.date(dt.strptime(str(x), "%Y-%m-%d %H:%M:%S")) )

In [4]:
data['count'] = [1 for i in range(len(data['date']))]

In [5]:
grouped = data.groupby(by = ['date', 'article_name'], as_index=False).sum()

In [6]:
grouped.drop([i for i in range(22)], inplace=True)

In [7]:
grouped.reset_index(inplace=True)

In [8]:
grouped.drop(columns=['index'], axis=1, inplace=True)

In [9]:
grouped.head()

,date,article_name,count
0,2018-05-29,Apfeltasche,6
1,2018-05-29,Brezel,33
2,2018-05-29,Buttercroissant,86
3,2018-05-30,Apfeltasche,10
4,2018-05-30,Brezel,65


In [10]:
print (len(grouped[grouped['article_name'] == 'Brezel']))
print (len(grouped[grouped['article_name'] == 'Buttercroissant']))
print (len(grouped[grouped['article_name'] == 'Apfeltasche']))

276
276
276


In [11]:
input_samples = []
temp = []
for i in range(len(grouped['count'])):
    temp.append(grouped['count'][i])
    if i % 3 == 2:
        input_samples.append(temp)
        temp = []
del(temp)

In [12]:
size = 7

In [13]:
train = input_samples[:int(len(input_samples) * 0.8)]
test = input_samples[int(len(input_samples) * 0.8):]

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

Using TensorFlow backend.


In [15]:
np.random.seed(10)

In [16]:
def create_X_Y(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        X.append(a)
        Y.append(dataset[i + look_back])
    return np.array(X), np.array(Y)

In [17]:
X_train, y_train = create_X_Y(train, size)
X_test, y_test = create_X_Y(test, size)

In [18]:
X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]*X_train.shape[2]), order='F')
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]*X_test.shape[2]), order='F')

In [33]:
model = Sequential()
model.add(Dense(21, input_shape=(size * 3,), activation='relu'))
# model.add(Dense(21, activation='relu'))
model.add(Dense(3))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train1, y_train, epochs=100, batch_size=3, verbose=2)

Epoch 1/100
 - 0s - loss: 5272.4074
Epoch 2/100
 - 0s - loss: 650.8610
Epoch 3/100
 - 0s - loss: 607.5950
Epoch 4/100
 - 0s - loss: 580.9463
Epoch 5/100
 - 0s - loss: 545.9153
Epoch 6/100
 - 0s - loss: 523.0063
Epoch 7/100
 - 0s - loss: 501.2307
Epoch 8/100
 - 0s - loss: 478.8108
Epoch 9/100
 - 0s - loss: 462.9621
Epoch 10/100
 - 0s - loss: 450.6629
Epoch 11/100
 - 0s - loss: 434.0765
Epoch 12/100
 - 0s - loss: 426.2109
Epoch 13/100
 - 0s - loss: 404.7053
Epoch 14/100
 - 0s - loss: 396.4696
Epoch 15/100
 - 0s - loss: 380.0647
Epoch 16/100
 - 0s - loss: 367.1547
Epoch 17/100
 - 0s - loss: 357.9441
Epoch 18/100
 - 0s - loss: 349.4953
Epoch 19/100
 - 0s - loss: 340.7374
Epoch 20/100
 - 0s - loss: 330.4969
Epoch 21/100
 - 0s - loss: 327.6078
Epoch 22/100
 - 0s - loss: 313.9693
Epoch 23/100
 - 0s - loss: 310.7643
Epoch 24/100
 - 0s - loss: 310.1434
Epoch 25/100
 - 0s - loss: 304.4410
Epoch 26/100
 - 0s - loss: 301.4548
Epoch 27/100
 - 0s - loss: 294.7001
Epoch 28/100
 - 0s - loss: 282.2285


In [34]:
trainScore = model.evaluate(X_train1, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(X_test1, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

Train Score: 210.77 MSE (14.52 RMSE)
Test Score: 224.27 MSE (14.98 RMSE)


In [35]:
def Predict_Submission(model, data, length):
    output = []
    for i in range(28):
        output.append(model.predict(np.reshape(data,(length, data.shape[1] * data.shape[2]), order='F'))[-1])
        data = np.array(list(data[1:]) + [(list(data[-1][1:]) + [output[-1]])])
    return np.array(output)

In [36]:
pred = Predict_Submission(model, X_test[-size*3:], size*3)

In [37]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
date               28 non-null object
Brezel             28 non-null float32
Buttercroissant    28 non-null float32
Apfeltasche        28 non-null float32
dtypes: float32(3), object(1)
memory usage: 640.0+ bytes


In [38]:
sample_submission['Apfeltasche'] = pred[:, 0]
sample_submission['Brezel'] = pred[:, 1]
sample_submission['Buttercroissant'] = pred[:, 2]

In [39]:
sample_submission

,date,Brezel,Buttercroissant,Apfeltasche
0,2019-03-01,89.964958,67.959366,12.728394
1,2019-03-02,73.152649,70.715195,12.641038
2,2019-03-03,74.171669,73.625679,11.425685
3,2019-03-04,82.574699,65.768898,13.544490
4,2019-03-05,71.180763,73.712952,11.295872
5,2019-03-06,75.867332,69.407471,12.444473
6,2019-03-07,70.226479,69.659210,11.495502
7,2019-03-08,77.300560,72.515915,12.099681
8,2019-03-09,78.660629,72.289139,13.107736
9,2019-03-10,76.224449,72.325493,11.669325


In [40]:
sample_submission.to_csv('answer.csv')